In [55]:
from openpyxl import Workbook

wb = Workbook()
ws = wb.active

In [5]:
created_sheet = wb.create_sheet(title='create_sheet')
print(created_sheet.title)

create_sheet


In [56]:
ws2 = wb.create_sheet("Mysheet")

In [57]:
ws3 = wb.create_sheet("My Last sheet", -1)

In [58]:
ws.title = "New Title"

In [59]:
ws1 = wb["New Title"]  # it's set before when you set ws.title

In [60]:
print(wb.sheetnames)

['New Title', 'My Last sheet', 'Mysheet']


In [61]:
for sh in wb:
    print(sh.title)

New Title
My Last sheet
Mysheet


In [62]:
# c = ws['A4']
ws1['A1'] = 'first'

In [63]:
cell_range = ws1['A2':'C3'] 
for i, row in enumerate(cell_range):
    for i, c in enumerate(row):
        print(i, c)
        c.value = i

0 <Cell 'New Title'.A2>
1 <Cell 'New Title'.B2>
2 <Cell 'New Title'.C2>
0 <Cell 'New Title'.A3>
1 <Cell 'New Title'.B3>
2 <Cell 'New Title'.C3>


In [64]:
wb.save('test_openpyxl.xlsx')

# deepbrain excel

In [4]:
import os 
from dataclasses import dataclass
import json
from openpyxl import Workbook
import datetime


@dataclass
class Patient:
    name: str
    pid: int
    sex: str
    age: int
    t2w_zip_path: str
    # WMH: str
    # WM: str
    # TIV: str
    t1w_zip_path:str
    # json_path:str
    # large_json_path: str
    # thick_json_path:str
    # shape_json_path:str


patient_list = [
    Patient(name='a', pid=1, sex='f', age=10, t2w_zip_path=None, t1w_zip_path=None),
    Patient(name='b', pid=2, sex='m', age=13, t2w_zip_path=None, t1w_zip_path=None),
    Patient(name='c', pid=3, sex='m', age=19, t2w_zip_path=None, t1w_zip_path=None)
]
user_id = 1

wb = Workbook(write_only=True)
sheet_list = []
sheet_list.append(wb.create_sheet(title='volume'))
sheet_list.append(wb.create_sheet(title='volume normative percentile'))
sheet_list.append(wb.create_sheet(title='ICV'))
sheet_list.append(wb.create_sheet(title='ICV normative percentile'))
sheet_list.append(wb.create_sheet(title='radiomics feature'))
sheet_list.append(wb.create_sheet(title='cortical thickness'))
sheet_list.append(wb.create_sheet(title='cortical thickness np'))
sheet_list.append(wb.create_sheet(title='wmh'))
# asset_path = os.path.join(base_path, BaseConfig.ASSET_PATH)
name_list = []
asset_path = './'

with open(os.path.join(asset_path, 'name.txt'), 'r') as f:
    for line in f.readlines():
        name_list.append(line.strip('\n').split(','))
header = ['Name', 'ID', 'Sex', 'Age', 'Birth Date', 'Study Date', 'Analysis Date']
shape_list = [
    'Sphericity',
    'Maximum3DDiameter',
    'LeastAxisLength',
    'Maximum2DDiameterColumn',
    'SurfaceArea',
    'Maximum2DDiameterSlice',
    'Elongation',
    'SurfaceVolumeRatio',
    'Flatness',
    'MajorAxisLength',
    'VoxelVolume',
    'MeshVolume',
    'MinorAxisLength',
    'Maximum2DDiameterRow'
]

shape_name_list = []
with open(os.path.join(asset_path, 'shape.txt'), 'r') as f:
    for line in f.readlines():
        shape_name_list.append(line.strip('\n'))

shape_header = ['Name', 'ID', 'Sex', 'Age', 'Birth Date', 'Study Date', 'Analysis Date']
for name in shape_name_list:
    for shape in shape_list:
        shape_header += [name + ' / ' + shape]
for name, _ in name_list:
    header += [name + ' left']
    header += [name + ' right']
    header += [name + ' total']
wmh_header = ['Name', 'ID', 'Sex', 'Age', 'Birth Date', 'Study Date', 'Analysis Date',
                'WMH volume', 'WM volume', 'TIV', '(WMH volume) / (WM volume)']
for i in range(8):
    if i == 4:
        sheet_list[i].append(shape_header)
    elif i == 7:
        sheet_list[i].append(wmh_header)
    else:
        sheet_list[i].append(header)
step = int(len(patient_list) / 10) if len(patient_list) > 10 else 1

for idx, patient in enumerate(patient_list):
    if idx % step == 0:
        print(f'{idx} % {step} send_socket for progress.......')
        # self._send_socket('export_doing', {
        #     'user_id': user_id,
        #     'percent': int((100 * idx) / len(patient_list))
        # })
    data_list = []
    for sheet in sheet_list:
        data_list.append([
            patient.name,
            patient.pid,
            patient.sex,
            patient.age,
            '', # patient.date_of_birth.strftime('%Y-%m-%d') if patient.date_of_birth is not None else '',
            '', # patient.date_of_study.strftime('%Y-%m-%d') if patient.date_of_study is not None else '',
            '' # patient.date_of_analysis.strftime('%Y-%m-%d') if patient.date_of_analysis is not None else ''
        ])
    if patient.t2w_zip_path is not None:
        WMH = patient.WMH
        WM = patient.WM
        TIV = patient.TIV
        data_list[7] += [
            WMH if WMH else '-',
            WM if WM else '-',
            TIV if TIV else '-',
            WMH / (WM / 1000) if WMH and WM else '-'
        ]
    else:
        data_list[7] += ['-', '-', '-', '-']
    if patient.t1w_zip_path is not None:
        old_json_data = json.load(open(patient.json_path, 'r'))
        large_data = json.load(open(patient.large_json_path, 'r'))
        old_ct_data = eval(json.load(open(patient.thick_json_path, 'r')))
        old_shape_data = json.load(open(patient.shape_json_path, 'r'))

        json_data = []
        idx_map = {}
        for i, r in enumerate(old_json_data):
            idx_map[r['brain_regions']] = ('small', i)
        for i, r in enumerate(large_data):
            idx_map[r['brain_regions']] = ('large', i)
        for _, key in name_list:
            text, i = idx_map[key]
            if text == 'small':
                json_data.append(old_json_data[i])
            else:
                json_data.append(large_data[i])

        ct_data = []
        idx_map = {}
        for i, r in enumerate(old_ct_data):
            idx_map[r['brain_regions']] = i
        for _, key in name_list:
            if key in idx_map:
                ct_data.append(old_ct_data[idx_map[key]])
            else:
                ct_data.append({
                    'brain_regions': key,
                    'cortical_thickness_left': -1,
                    'cortical_thickness_right': -1,
                    'cortical_thickness_total': -1,
                    'np_cortical_thickness_left': -1,
                    'np_cortical_thickness_right': -1,
                    'np_cortical_thickness_total': -1
                })
        shape_data = []
        idx_map = {}
        for i, r in enumerate(old_shape_data):
            idx_map[r['brain_regions']] = i
        for name in shape_name_list:
            if name in idx_map:
                shape_data.append(old_shape_data[idx_map[name]])
            else:
                temp = {}
                for shape in shape_list:
                    temp[shape] = '-'
                shape_data.append(temp)

        for part in json_data:
            if part['brain_regions'] == 'vessel':
                continue
            data_list[0] += [part['volume_left'] / 1000] if part['volume_left'] >= 0 else ['-']
            data_list[0] += [part['volume_right'] / 1000] if part['volume_right'] >= 0 else ['-']
            data_list[0] += [part['volume_total'] / 1000] if part['volume_total'] >= 0 else ['-']
            data_list[1] += [part['np_volume_left']] if part['np_volume_left'] >= 0 else ['-']
            data_list[1] += [part['np_volume_right']] if part['np_volume_right'] >= 0 else ['-']
            data_list[1] += [part['np_volume_total']] if part['np_volume_total'] >= 0 else ['-']
            data_list[2] += [part['icv_left'] * 100] if part['icv_left'] >= 0 else ['-']
            data_list[2] += [part['icv_right'] * 100] if part['icv_right'] >= 0 else ['-']
            data_list[2] += [part['icv_total'] * 100] if part['icv_total'] >= 0 else ['-']
            data_list[3] += [part['np_icv_left']] if part['np_icv_left'] >= 0 else ['-']
            data_list[3] += [part['np_icv_right']] if part['np_icv_right'] >= 0 else ['-']
            data_list[3] += [part['np_icv_total']] if part['np_icv_total'] >= 0 else ['-']
        for part in ct_data:
            if part['brain_regions'] == 'vessel':
                continue
            data_list[5] += [part['cortical_thickness_left']
                                ] if part['cortical_thickness_left'] >= 0 else ['-']
            data_list[5] += [part['cortical_thickness_right']
                                ] if part['cortical_thickness_right'] >= 0 else ['-']
            data_list[5] += [part['cortical_thickness_total']
                                ] if part['cortical_thickness_total'] >= 0 else ['-']
            data_list[6] += [part['np_cortical_thickness_left']
                                ] if part['np_cortical_thickness_left'] >= 0 else ['-']
            data_list[6] += [part['np_cortical_thickness_right']
                                ] if part['np_cortical_thickness_right'] >= 0 else ['-']
            data_list[6] += [part['np_cortical_thickness_total']
                                ] if part['np_cortical_thickness_total'] >= 0 else ['-']

        for shape_dic in shape_data:
            for shape in shape_list:
                data_list[4] += [shape_dic[shape]]
    else:
        for i in range(8):
            if i == 4:
                data_list[i] += ['-'] * 224
            elif i != 7:
                data_list[i] += ['-'] * 189
    for i in range(len(data_list)):
        sheet_list[i].append(data_list[i])
excel_name = str(user_id) + '_' + \
    datetime.datetime.now().strftime("%Y%m%d_%H%M%S.%f") + '.xlsx'
excel_path = os.path.join(asset_path, excel_name)
wb.save(excel_path)
print(excel_path)

0 % 1 send_socket for progress.......
1 % 1 send_socket for progress.......
2 % 1 send_socket for progress.......
./1_20221221_132727.325322.xlsx


In [22]:
import os
import json
import datetime
from openpyxl import Workbook
# from core.const.enum import BaseEnum
from enum import EnumMeta


class excel_sheet_title(EnumMeta):
    volumn = 'volumn'
    volumn_normative = 'volume normative percentile'
    ICV = 'ICV'
    ICV_normative = 'ICV normative percentile'
    radiomics = 'radiomics feature'
    cortical = 'cortical thickness'
    cortical_np = 'cortical thickness np'
    wmh = 'wmh'

print(excel_sheet_title.volumn)
wb = Workbook(write_only=True)
wb_volume = wb.create_sheet(title=excel_sheet_title.volumn)
wb_volumn_normative = wb.create_sheet(title=excel_sheet_title.volumn_normative)
wb_ICV = wb.create_sheet(title=excel_sheet_title.ICV)

sheets = [wb_volume, wb_volumn_normative, wb_ICV]

for i, sheet in enumerate(sheets):
    headers = [f'header {i}']*10
    sheet.append(headers)

# for sheet in sheets:
#     sheet.append(['info']*3)

count_defualt = len(headers) - len(patient)

print(headers, len(headers))
print(patient, len(patient))
print(count_defualt)

for i in range(0, 20):
    patient = ['info']*3
    default_data = ['-'] * count_defualt
    wb_volume.append(patient + list(range(7)))
    wb_volumn_normative.append(patient + default_data)
    wb_ICV.append(patient + list(range(7)))


wb.save('test_openpyxl.xlsx')

volumn
['header 2', 'header 2', 'header 2', 'header 2', 'header 2', 'header 2', 'header 2', 'header 2', 'header 2', 'header 2'] 10
['info', 'info', 'info'] 3
7


In [19]:
a = list(range(5))
b = a.copy()
b.extend(list(range(5)))
a, b

([0, 1, 2, 3, 4], [0, 1, 2, 3, 4, 0, 1, 2, 3, 4])

In [21]:
[1,2] + [1,3]

[1, 2, 1, 3]

In [24]:
import os 
from dataclasses import dataclass
import json
from openpyxl import Workbook
import datetime
from enum import EnumMeta


@dataclass
class Patient:
    name: str
    pid: int
    sex: str
    age: int
    t2w_zip_path: str
    # WMH: str
    # WM: str
    # TIV: str
    t1w_zip_path:str
    # json_path:str
    # large_json_path: str
    # thick_json_path:str
    # shape_json_path:str


patient_list = [
    Patient(name='a', pid=1, sex='f', age=10, t2w_zip_path=None, t1w_zip_path=None),
    Patient(name='b', pid=2, sex='m', age=13, t2w_zip_path=None, t1w_zip_path=None),
    Patient(name='c', pid=3, sex='m', age=19, t2w_zip_path=None, t1w_zip_path=None)
]
user_id = 1


class ExcelSheetTitles(EnumMeta):
    volumn = 'volumn'
    volumn_normative = 'volume normative percentile'
    ICV = 'ICV'
    ICV_normative = 'ICV normative percentile'
    radiomics = 'radiomics feature'
    cortical = 'cortical thickness'
    cortical_np = 'cortical thickness np'
    wmh = 'wmh'


class ExcelResult:
    def __init__(self, user_id, patient_list):
        assert patient_list, 'patient list should not be empty!'
        self.user_id = user_id
        self.patient_list = patient_list

    def _make_json_data(self, old_json_data, large_data, common_study_data_key):
        json_data = []
        idx_map = {}
        for i, r in enumerate(old_json_data):
            idx_map[r['brain_regions']] = ('small', i)
        for i, r in enumerate(large_data):
            idx_map[r['brain_regions']] = ('large', i)
        for key in common_study_data_key:
            text, i = idx_map[key]
            if text == 'small':
                json_data.append(old_json_data[i])
            else:
                json_data.append(large_data[i])
        return json_data

    def _make_ct_data(self, old_ct_data, common_study_data_key):
        ct_data = []
        idx_map = {}
        for i, r in enumerate(old_ct_data):
            idx_map[r['brain_regions']] = i
        for key in common_study_data_key:
            if key in idx_map:
                ct_data.append(old_ct_data[idx_map[key]])
            else:
                ct_data.append({
                    'brain_regions': key,
                    'cortical_thickness_left': -1,
                    'cortical_thickness_right': -1,
                    'cortical_thickness_total': -1,
                    'np_cortical_thickness_left': -1,
                    'np_cortical_thickness_right': -1,
                    'np_cortical_thickness_total': -1
                })
        return ct_data

    def _make_shape_data(self, old_shape_data, radiomics_shape_list_group1, radiomics_shape_list_group2):
        shape_data = []
        idx_map = {}
        for i, r in enumerate(old_shape_data):
            idx_map[r['brain_regions']] = i
        for name in radiomics_shape_list_group1:
            if name in idx_map:
                shape_data.append(old_shape_data[idx_map[name]])
            else:
                temp = {}
                for shape in radiomics_shape_list_group2:
                    temp[shape] = '-'
                shape_data.append(temp)
        return shape_data

    def create_excel_result(self):
        wb = Workbook(write_only=True)
        wb_volume = wb.create_sheet(title=ExcelSheetTitles.volumn)
        wb_volumn_normative = wb.create_sheet(title=ExcelSheetTitles.volumn_normative)
        wb_ICV = wb.create_sheet(title=ExcelSheetTitles.ICV)
        wb_ICV_normative = wb.create_sheet(title=ExcelSheetTitles.ICV_normative)
        wb_radiomics = wb.create_sheet(title=ExcelSheetTitles.radiomics)
        wb_cortical = wb.create_sheet(title=ExcelSheetTitles.cortical)
        wb_cortical_np = wb.create_sheet(title=ExcelSheetTitles.cortical_np)
        wb_wmh = wb.create_sheet(title=ExcelSheetTitles.wmh)

        # ======== making haader =========

        asset_path = './'  # FIXME
        # asset_path = config.ASSET_PATH

        common_study_data_name = []
        common_study_data_key = []
        with open(os.path.join(asset_path, 'name.txt'), 'r') as f:
            for line in f.readlines():
                name, key = line.strip('\n').split(',')
                common_study_data_name.append(name)
                common_study_data_key.append(key)

        header_common = ['Name', 'ID', 'Sex', 'Age', 'Birth Date', 'Study Date', 'Analysis Date']

        # common header
        full_header_common = header_common.copy()
        for shape_name in common_study_data_name:
            full_header_common += [shape_name + ' left']
            full_header_common += [shape_name + ' right']
            full_header_common += [shape_name + ' total']

        common_sheet_list = [
            wb_volume, wb_volumn_normative, wb_ICV, wb_ICV_normative, wb_cortical, wb_cortical_np
        ]
        for sheet in common_sheet_list:
            sheet.append(full_header_common)

        # radiomics feature header
        radiomics_shape_list_group1 = []
        with open(os.path.join(asset_path, 'shape.txt'), 'r') as f:
            for line in f.readlines():
                radiomics_shape_list_group1.append(line.strip('\n'))

        radiomics_shape_list_group2 = [
            'Sphericity',
            'Maximum3DDiameter',
            'LeastAxisLength',
            'Maximum2DDiameterColumn',
            'SurfaceArea',
            'Maximum2DDiameterSlice',
            'Elongation',
            'SurfaceVolumeRatio',
            'Flatness',
            'MajorAxisLength',
            'VoxelVolume',
            'MeshVolume',
            'MinorAxisLength',
            'Maximum2DDiameterRow'
        ]

        full_header_radiomics = header_common.copy()
        for prefix in radiomics_shape_list_group1:
            for subfix in radiomics_shape_list_group2:
                full_header_radiomics += [prefix + ' / ' + subfix]

        wb_radiomics.append(full_header_radiomics)

        # wmh header
        full_wmh_header = header_common.copy() + ['WMH volume', 'WM volume', 'TIV', '(WMH volume) / (WM volume)']
        wb_wmh.append(full_wmh_header)

        step = int(len(self.patient_list) / 10) if len(self.patient_list) > 10 else 1

        # ======== filling cell data ========

        for idx, patient in enumerate(self.patient_list):
            if idx % step == 0:
                print(f'{idx} % {step} send_socket for progress.......')
                # FIXME
                # self._send_socket('export_doing', {
                #     'user_id': user_id,
                #     'percent': int((100 * idx) / len(patient_list))
                # })

            patient_info = [
                patient.name,
                patient.pid,
                patient.sex,
                patient.age,
                '', # patient.date_of_birth.strftime('%Y-%m-%d') if patient.date_of_birth is not None else '',
                '', # patient.date_of_study.strftime('%Y-%m-%d') if patient.date_of_study is not None else '',
                '', # patient.date_of_analysis.strftime('%Y-%m-%d') if patient.date_of_analysis is not None else ''
            ]

            volume_study_data = []
            volume_normative_study_data = []
            icv_study_data = []
            icv_normative_study_data = []
            radiomics_study_data = []
            cortical_study_data = []
            cortical_np_study_data = []
            wmh_study_data = []

            if patient.t1w_zip_path:
                old_json_data = json.load(open(patient.json_path, 'r'))
                large_data = json.load(open(patient.large_json_path, 'r'))
                old_ct_data = eval(json.load(open(patient.thick_json_path, 'r')))
                old_shape_data = json.load(open(patient.shape_json_path, 'r'))

                json_data = self._make_json_data(old_json_data, large_data, common_study_data_key)
                shape_data = self._make_shape_data(old_shape_data, radiomics_shape_list_group1, radiomics_shape_list_group2)
                ct_data = self._make_ct_data(old_ct_data, common_study_data_key)

                for part in json_data:
                    if part['brain_regions'] == 'vessel':
                        continue

                    volume_study_data += [part['volume_left'] / 1000] if part['volume_left'] >= 0 else ['-']
                    volume_study_data += [part['volume_right'] / 1000] if part['volume_right'] >= 0 else ['-']
                    volume_study_data += [part['volume_total'] / 1000] if part['volume_total'] >= 0 else ['-']

                    volume_normative_study_data += [part['np_volume_left']] if part['np_volume_left'] >= 0 else ['-']
                    volume_normative_study_data += [part['np_volume_right']] if part['np_volume_right'] >= 0 else ['-']
                    volume_normative_study_data += [part['np_volume_total']] if part['np_volume_total'] >= 0 else ['-']

                    icv_study_data += [part['icv_left'] * 100] if part['icv_left'] >= 0 else ['-']
                    icv_study_data += [part['icv_right'] * 100] if part['icv_right'] >= 0 else ['-']
                    icv_study_data += [part['icv_total'] * 100] if part['icv_total'] >= 0 else ['-']

                    icv_normative_study_data += [part['np_icv_left']] if part['np_icv_left'] >= 0 else ['-']
                    icv_normative_study_data += [part['np_icv_right']] if part['np_icv_right'] >= 0 else ['-']
                    icv_normative_study_data += [part['np_icv_total']] if part['np_icv_total'] >= 0 else ['-']

                for shape_dic in shape_data:
                    for shape in radiomics_shape_list_group2:
                        radiomics_study_data += [shape_dic[shape]]

                for part in ct_data:
                    if part['brain_regions'] == 'vessel':
                        continue

                    cortical_study_data += [
                        part['cortical_thickness_left']
                    ] if part['cortical_thickness_left'] >= 0 else ['-']
                    cortical_study_data += [
                        part['cortical_thickness_right']
                    ] if part['cortical_thickness_right'] >= 0 else ['-']
                    cortical_study_data += [
                        part['cortical_thickness_total']
                    ] if part['cortical_thickness_total'] >= 0 else ['-']

                    cortical_np_study_data += [
                        part['np_cortical_thickness_left']
                    ] if part['np_cortical_thickness_left'] >= 0 else ['-']
                    cortical_np_study_data += [
                        part['np_cortical_thickness_right']
                    ] if part['np_cortical_thickness_right'] >= 0 else ['-']
                    cortical_np_study_data += [
                        part['np_cortical_thickness_total']
                    ] if part['np_cortical_thickness_total'] >= 0 else ['-']
            else:
                # no t1w_zip_path to set '-'

                count_common_data = len(full_header_common) - len(patient_info)
                count_raiomics_data = len(full_header_radiomics) - len(patient_info)

                volume_study_data = ['-'] * count_common_data
                volume_normative_study_data = ['-'] * count_common_data
                icv_study_data = ['-'] * count_common_data
                icv_normative_study_data = ['-'] * count_common_data
                cortical_study_data = ['-'] * count_common_data
                cortical_np_study_data = ['-'] * count_common_data

                radiomics_study_data = ['-'] * count_raiomics_data

            if patient.t2w_zip_path:
                WMH = patient.WMH
                WM = patient.WM
                TIV = patient.TIV
                wmh_study_data += [
                    WMH if WMH else '-',
                    WM if WM else '-',
                    TIV if TIV else '-',
                    WMH / (WM / 1000) if WMH and WM else '-'
                ]
            else:
                # no t2w_zip_path to set '-'
                count_wmh_data = len(full_wmh_header) - len(patient_info)
                wmh_study_data = ['-'] * count_wmh_data

            wb_volume.append(patient_info + volume_study_data)
            wb_volumn_normative.append(patient_info + volume_normative_study_data)
            wb_ICV.append(patient_info + icv_study_data)
            wb_ICV_normative.append(patient_info + icv_normative_study_data)
            wb_radiomics.append(patient_info + radiomics_study_data)
            wb_cortical.append(patient_info + cortical_study_data)
            wb_cortical_np.append(patient_info + cortical_np_study_data)
            wb_wmh.append(patient_info + wmh_study_data)

        excel_name = str(self.user_id) + '_' + \
            datetime.datetime.now().strftime("%Y%m%d_%H%M%S.%f") + '.xlsx'
        excel_path = os.path.join(asset_path, excel_name)
        wb.save(excel_path)


ExcelResult(user_id, patient_list).create_excel_result()

0 % 1 send_socket for progress.......
1 % 1 send_socket for progress.......
2 % 1 send_socket for progress.......


In [26]:
(1,) + (1,2)

(1, 1, 2)